# Pull PVWatts Data into Excel Sheet

In [1]:
import requests
import json
import openpyxl
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
import calendar
import datetime
from statistics import mean

import config

In [2]:
#Select address or lon/lat
address = "Honolulu"
lat = 37.77     #Range -90 to 90
lon = -122.42     #Rangel -180 to 180

#Select true if using lon/lat coordinates or false to use address
uselonlat = False

#Tilt angle (Range 0-90, 0 = horizontal, 90 = vertical, 20 = typical tracking array)
#tilt = 0
tiltvals = [0,20,45,90]

#Azimuth (Range 0-360, 0=N, 45=NE, 90=E, 135=SE, 180=S, 225=SW, 270=W, 315=NW)
#azimuth = 180
azimuthvals = [0,45,90,135,180,225,270,315]
# azimuthvals = [0,90,180,270]

#Name of file locatin to save data
fname = "pvwattsdata.xlsx"

In [3]:
def azimuthtitle(a):
    switcher = {
        0:'N',
        45:'NE', 
        90:'E', 
        135:'SE', 
        180:'S', 
        225:'SW', 
        270:'W', 
        315:'NW' 
    }
    return switcher.get(a,a)

In [4]:
##########Set up workbook to write data into
wb = Workbook()
ws = wb.active
ws.title = "PVWatts"
rshift = 18 #number of rows to shift data for new location

In [5]:
#loop through tilts
for i in range(len(tiltvals)):
    
    tilt = tiltvals[i]
    
    ws.cell(7+i*rshift,1).value = f"Tilt {tilt}\N{DEGREE SIGN}" 
    ws.cell(8+i*rshift,2).value = "kWh/m2/day"
    for m in range(12):
        ws.cell(9+i*rshift+m,2).value = calendar.month_name[m+1]
    ws.cell(21+i*rshift,2).value = "Average"
    ws.cell(22+i*rshift,2).value = "-"
    ws.cell(23+i*rshift,2).value = "+"
    
    #loop through angles
    for j in range(len(azimuthvals)):
        azimuth = azimuthvals[j]
        az = azimuthtitle(azimuth)
        ws.cell(8+i*rshift,3+j).value = az
    
        ##########Query PVWatts to get solar data
        url = "https://developer.nrel.gov/api/pvwatts/v6.json"
        payload = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"\"\r\n\r\n\r\n-----011000010111000001101001--\r\n"
        headers = {'content-type': 'multipart/form-data; boundary=---011000010111000001101001'}
        if uselonlat == True:
            querystring = {"api_key":config.api_key,"system_capacity":"4",
                           "module_type":"0","losses":"10","array_type":"0","tilt":tilt,"azimuth":azimuth,
                           "lat":lat,"dataset":"tmy3","lon":lon}
        else:
            querystring = {"api_key":config.api_key,"system_capacity":"4",
                           "module_type":"0","losses":"10","array_type":"0","tilt":tilt,"azimuth":azimuth,
                           "address":address,"dataset":"tmy3"}
        response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

        ###########Solar data and used data parsing from output of query
        #print(response.text)
        sm = json.loads(response.text)['outputs']['solrad_monthly']
        #print(sm)
        for m in range(12):
            ws.cell(9+i*rshift+m,3+j).value = round(sm[m],2) 
        ws.cell(21+i*rshift,3+j).value = round(mean(sm),2) 
        ws.cell(22+i*rshift,3+j).value = round(mean(sm)-min(sm),2)
        ws.cell(23+i*rshift,3+j).value = round(max(sm)-mean(sm),2)  
    
        if tilt == 0:
            ws.cell(8+i*rshift,3+j).value = "Horizontal"
            break


In [6]:
#parse out the latitude, longitude, and city location used for analysis
used_lat = json.loads(response.text)['station_info']['lat']
used_lon = json.loads(response.text)['station_info']['lon']
if uselonlat == False:
    used_address = address
else:
    used_address = json.loads(response.text)['station_info']['city'] + ", " + \
                json.loads(response.text)['station_info']['state']

#format the 
ws.cell(row=3,column=1).value = f"Location: {used_address}"
ws.cell(row=4,column=1).value = f"Latitude: {used_lat:.2f}"
ws.cell(row=5,column=1).value = f"Longitude: {used_lon:.2f}"

ws.column_dimensions["A"].width = 30
ws.column_dimensions["B"].width = 12
ws.column_dimensions["C"].width = 6
    
wb.save(fname)